In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd

# Define the neural network architecture
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden_layers = nn.Sequential(
            nn.Linear(4, 50),  # 4 initial parameter (x, y, z,t)
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 3)  # 3 output paramemter (u, v, w)
        )

    def forward(self, x):
        return self.hidden_layers(x)
print("i can do anything, and i will do")

i can do anything, and i will do


In [ ]:
import numpy as np

# Initial condition: u(x, y, z, t=0) = v(x, y, z, t=0) = w(x, y, z, t=0) = 0
def initial_condition(x, y, z):
    return np.zeros((x.shape[0], 3))

# Boundary condition: u, v, w = 0 at the boundaries
def boundary_condition(x, y, z, t):
    return np.zeros((x.shape[0], 3))


In [ ]:
def burgers_pde_residual(model, x, y, z, t, nu):
    X = torch.cat((x, y, z, t), dim=1)
    X.requires_grad = True  # Set requires_grad for backpropagation
    #define nu
    nu = 0.01

    u, v, w = model(X).split(1, dim=1)  # Predict velocity components


    u_x = autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_y = autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_z = autograd.grad(u, z, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_t = autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    v_x = autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_y = autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_z = autograd.grad(v, z, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_t = autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    w_x = autograd.grad(w, x, grad_outputs=torch.ones_like(w), create_graph=True)[0]
    w_y = autograd.grad(w, y, grad_outputs=torch.ones_like(w), create_graph=True)[0]
    w_z = autograd.grad(w, z, grad_outputs=torch.ones_like(w), create_graph=True)[0]
    w_t = autograd.grad(w, t, grad_outputs=torch.ones_like(w), create_graph=True)[0]

    u_xx = autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_yy = autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
    u_zz = autograd.grad(u_z, z, grad_outputs=torch.ones_like(u_z), create_graph=True)[0]
    v_xx = autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
    v_yy = autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]
    v_zz = autograd.grad(v_z, z, grad_outputs=torch.ones_like(v_z), create_graph=True)[0]
    w_xx = autograd.grad(w_x, x, grad_outputs=torch.ones_like(w_x), create_graph=True)[0]
    w_yy = autograd.grad(w_y, y, grad_outputs=torch.ones_like(w_y), create_graph=True)[0]
    w_zz = autograd.grad(w_z, z, grad_outputs=torch.ones_like(w_z), create_graph=True)[0]

    res_u = u_t + u * u_x + v * u_y + w * u_z - nu * (u_xx + u_yy + u_zz)
    res_v = v_t + u * v_x + v * v_y + w * v_z - nu * (v_xx + v_yy + v_zz)
    res_w = w_t + u * w_x + v * w_y + w * w_z - nu * (w_xx + w_yy + w_zz)

    return res_u, res_v, res_w

print("d")


d


In [ ]:
#define the model, optimizer, and loss function
import torch.optim as optim
import torch.nn as nn
loss_fn = nn.MSELoss()
#given value
nu = 0.01


model = PINN()
optimizer = optim.Adam(model.parameters(), lr = 0.01)


In [ ]:
import torch
#train the model
# Define the ranges for each dimension
x = torch.linspace(-1, 1, 20)
y = torch.linspace(-1, 1, 20)
z = torch.linspace(-1, 1, 20)
t = torch.linspace(0, 1, 10)

# Create the meshgrid
x_train, y_train, z_train, t_train = torch.meshgrid(x, y, z, t, indexing='ij')

# Reshape the meshgrid tensors into column vectors
x_train = x_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
z_train = z_train.reshape(-1, 1)
t_train = t_train.reshape(-1, 1)

# Combine the coordinate tensors into a single dataset
train_data = torch.cat([x_train, y_train, z_train, t_train], dim=1)

print("Training data shape:", train_data.shape)


Training data shape: torch.Size([80000, 4])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

num_epochs = 12000
print("Starting Training...")
nu = 0.01

for epoch in range(num_epochs):
    model.train()

    # Compute the residuals at points
    res_u, res_v, res_w = burgers_pde_residual(model, x, y, z, t, nu)
    loss_res = torch.mean(res_u**2 + res_v**2 + res_w**2)

    # Compute the loss for initial conditions
    u_pred_init = model(torch.cat([x, y, z], dim=1))
    loss_init = nn.MSELoss()(u_pred_init[:, 0], x) + \
                nn.MSELoss()(u_pred_init[:, 1], y) + \
                nn.MSELoss()(u_pred_init[:, 2], z)

    # Compute the loss for boundary conditions
    u_pred_bound = model(torch.cat([x_bound, y_bound, z_bound, t_bound], dim=1))
    loss_bound = nn.MSELoss()(u_pred_bound[:, 0], u_bound) + \
                 nn.MSELoss()(u_pred_bound[:, 1], v_bound) + \
                 nn.MSELoss()(u_pred_bound[:, 2], w_bound)

    # Total loss
    loss_total = loss_res + loss_init + loss_bound

    # Backpropagation
    optimizer.zero_grad()
    loss_total.backward()
    optimizer.step()

    # Print loss every 1000 epochs
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss_total.item()}")

print("Training Complete!")


Starting Training...


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
import torch
import torch.nn as nn

num_epochs = 12000
print("d")
for each in range(num_epochs):
    model.train()

    # Compute the residuals at points
    res_u, res_v, res_w = burgers_pde_residual(model, x, y, z, t, nu)
    loss_res = torch.mean(res_u**2 + res_v**2 + res_w**2)
    # Compute the loss for initial conditions
    u_pred_init = model(torch.cat([x, y, z], dim=1))
    loss_init = nn.MSELoss()(u_pred_init[:, 0], x) + \
                nn.MSELoss()(u_pred_init[:, 1], y) + \
                nn.MSELoss()(u_pred_init[:, 2], z)

    # Compute the loss for boundary conditions
    u_pred_bound = model(torch.cat([x_bound, y_bound, z_bound, t_bound], dim=1))
    loss_bound = nn.MSELoss()(u_pred_bound[:, 0], u_bound) + \
                 nn.MSELoss()(u_pred_bound[:, 1], v_bound) + \
                 nn.MSELoss()(u_pred_bound[:, 2], w_bound)

    # Total loss
    loss_total = loss_res + loss_init + loss_bound



d


NameError: name 'model' is not defined

In [ ]:
import torch.optim as optim

# Hyperparameters
nu = 0.01  # Viscosity
epochs = 10000
learning_rate = 0.001

# Initialize model and optimizer
model = PINN()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = compute_loss(model, x_colloc, y_colloc, z_colloc, t_colloc,
                        x_init, y_init, z_init, t_init, u_init, v_init, w_init,
                        x_bound, y_bound, z_bound, t_bound, u_bound, v_bound, w_bound, nu)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")


NameError: name 'x_colloc' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np  # Added numpy import

# Define the PDE residual function
def pde_residual(x, y, z, t, model, nu=0.01):
    X = torch.cat((x, y, z, t), dim=1)
    X.requires_grad = True  # Set requires_grad for backpropagation

    u, v, w = model(X).split(1, dim=1)  # Predict velocity components

    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_z = torch.autograd.grad(u, z, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_zz = torch.autograd.grad(u_z, z, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_z = torch.autograd.grad(v, z, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_zz = torch.autograd.grad(v_z, z, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    w_t = torch.autograd.grad(w, t, grad_outputs=torch.ones_like(w), create_graph=True)[0]
    w_x = torch.autograd.grad(w, x, grad_outputs=torch.ones_like(w), create_graph=True)[0]
    w_y = torch.autograd.grad(w, y, grad_outputs=torch.ones_like(w), create_graph=True)[0]
    w_z = torch.autograd.grad(w, z, grad_outputs=torch.ones_like(w), create_graph=True)[0]

    w_xx = torch.autograd.grad(w_x, x, grad_outputs=torch.ones_like(w), create_graph=True)[0]
    w_yy = torch.autograd.grad(w_y, y, grad_outputs=torch.ones_like(w), create_graph=True)[0]
    w_zz = torch.autograd.grad(w_z, z, grad_outputs=torch.ones_like(w), create_graph=True)[0]

    # Burgers' equation residuals for each velocity component
    f_u = u_t + u * u_x + v * u_y + w * u_z - nu * (u_xx + u_yy + u_zz)
    f_v = v_t + u * v_x + v * v_y + w * v_z - nu * (v_xx + v_yy + v_zz)
    f_w = w_t + u * w_x + v * w_y + w * w_z - nu * (w_xx + w_yy + w_zz)

    return f_u, f_v, f_w


# Define Initial Condition Points
num_ic = 100  # Number of initial condition points
x_ic = torch.rand(num_ic, 1) * 2 - 1  # Random points in [-1,1]
y_ic = torch.rand(num_ic, 1) * 2 - 1
z_ic = torch.rand(num_ic, 1) * 2 - 1
t_ic = torch.zeros(num_ic, 1)  # Initial time t=0

# Define initial velocity conditions
u_ic = torch.sin(np.pi * x_ic) * torch.sin(np.pi * y_ic) * torch.sin(np.pi * z_ic)  # Example IC
v_ic = torch.zeros_like(u_ic)
w_ic = torch.zeros_like(u_ic)


# Initial and Boundary Condition Functions
def initial_condition(x, y, z):
    u = np.sin(np.pi * x) * np.sin(np.pi * y) * np.sin(np.pi * z)
    v = 0
    w = 0
    return torch.tensor([u, v, w], dtype=torch.float32)  # Convert to tensor


def boundary_condition(x, y, z, t):
    u = np.sin(np.pi * x) * np.sin(np.pi * y) * np.sin(np.pi * z) * np.cos(np.pi * t)
    v = 0
    w = 0
    return torch.tensor([u, v, w], dtype=torch.float32)  # Convert to tensor


print("kumar")  # Print statement is fine


kumar


2


In [ ]:
# train the model
import torch
# Define the training domain
x = torch.linspace(-1, 1, 200)
y = torch.linspace(-1, 1, 200)
z = torch.linspace(-1, 1, 200)
t = torch.linspace(0, 1, 100)
# Instead of creating a full meshgrid (which is too large), randomly sample points
num_samples = 100000  # Choose a reasonable number of points
x_train = torch.rand(num_samples, 1) * 2 - 1  # Scale to [-1, 1]
y_train = torch.rand(num_samples, 1) * 2 - 1
z_train = torch.rand(num_samples, 1) * 2 - 1
t_train = torch.rand(num_samples, 1)  # Scale to [0, 1]

print("hi")




hi


In [ ]:
#define the model, optimizer, and loss function
import torch.optim as optim
import torch.nn as nn
loss_fn = nn.MSELoss()


model = PINN()
optimizer = optim.Adam(model.parameters(), lr = 0.01)


In [ ]:
def loss_function(model, x_train, y_train, z_train, t_train,
                  x_ic, y_ic, z_ic, t_ic, u_ic, v_ic, w_ic,
                  x_bc, y_bc, z_bc, t_bc):

    # Compute network output for interior points
    u_pred, v_pred, w_pred = model(x, y, z, t)

    # Compute network output for initial condition points
    u_ic_pred, v_ic_pred, w_ic_pred = model(x_ic, y_ic, z_ic, t_ic)

    # Compute loss for initial conditions
    loss_ic = torch.mean((u_ic_pred - u_ic) ** 2) + \
              torch.mean((v_ic_pred - v_ic) ** 2) + \
              torch.mean((w_ic_pred - w_ic) ** 2)

    # Compute network output for boundary condition points
    u_bc_pred, v_bc_pred, w_bc_pred = model(x_bc, y_bc, z_bc, t_bc)

    # Compute loss for boundary conditions (periodic BC)
    loss_bc = torch.mean((u_bc_pred - u_bc_pred.detach()) ** 2) + \
              torch.mean((v_bc_pred - v_bc_pred.detach()) ** 2) + \
              torch.mean((w_bc_pred - w_bc_pred.detach()) ** 2)


    # PDE residual loss
    f_u, f_v, f_w = pde_residual(x, y, t, model, nu)
    loss_pde = torch.mean(f_u**2 + f_v**2 + f_w**2)

    loss = loss_ic + loss_bc + loss_pde
    return loss




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the PINN Model
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 3)  # Output: [u, v, w]
        )

    def forward(self, X):
        return self.net(X)

# Define the PDE residual function
def pde_residual(x, y, z, t, model, nu=0.01):
    # Ensure inputs require gradients
    x, y, z, t = [var.detach().requires_grad_() for var in [x, y, z, t]]

    X = torch.cat((x, y, z, t), dim=1)

    # Compute outputs
    u, v, w = model(X).split(1, dim=1)

    # Compute derivatives using autograd
    def gradient(y, x):
        return torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y), create_graph=True)[0]

    u_t, u_x, u_y, u_z = gradient(u, t), gradient(u, x), gradient(u, y), gradient(u, z)
    v_t, v_x, v_y, v_z = gradient(v, t), gradient(v, x), gradient(v, y), gradient(v, z)
    w_t, w_x, w_y, w_z = gradient(w, t), gradient(w, x), gradient(w, y), gradient(w, z)

    u_xx, u_yy, u_zz = gradient(u_x, x), gradient(u_y, y), gradient(u_z, z)
    v_xx, v_yy, v_zz = gradient(v_x, x), gradient(v_y, y), gradient(v_z, z)
    w_xx, w_yy, w_zz = gradient(w_x, x), gradient(w_y, y), gradient(w_z, z)

    # Burgers' equation residuals
    f_u = u_t + u * u_x + v * u_y + w * u_z - nu * (u_xx + u_yy + u_zz)
    f_v = v_t + u * v_x + v * v_y + w * v_z - nu * (v_xx + v_yy + v_zz)
    f_w = w_t + u * w_x + v * w_y + w * w_z - nu * (w_xx + w_yy + w_zz)

    return f_u, f_v, f_w

# Training domain
num_samples = 100000
x_train = torch.rand(num_samples, 1) * 2 - 1
y_train = torch.rand(num_samples, 1) * 2 - 1
z_train = torch.rand(num_samples, 1) * 2 - 1
t_train = torch.rand(num_samples, 1)

# Define Initial Condition Points
num_ic = 100
x_ic = torch.rand(num_ic, 1) * 2 - 1
y_ic = torch.rand(num_ic, 1) * 2 - 1
z_ic = torch.rand(num_ic, 1) * 2 - 1
t_ic = torch.zeros(num_ic, 1)

# Initial velocity conditions
u_ic = torch.sin(np.pi * x_ic) * torch.sin(np.pi * y_ic) * torch.sin(np.pi * z_ic)
v_ic = torch.zeros_like(u_ic)
w_ic = torch.zeros_like(u_ic)

# Define boundary points
num_bc = 100
x_bc = torch.rand(num_bc, 1) * 2 - 1
y_bc = torch.rand(num_bc, 1) * 2 - 1
z_bc = torch.rand(num_bc, 1) * 2 - 1
t_bc = torch.rand(num_bc, 1)

# Define loss function
def loss_function(model, x_train, y_train, z_train, t_train,
                  x_ic, y_ic, z_ic, t_ic, u_ic, v_ic, w_ic,
                  x_bc, y_bc, z_bc, t_bc):

    # Concatenate inputs before passing to the model
    X_train = torch.cat((x_train, y_train, z_train, t_train), dim=1).requires_grad_()
    X_ic = torch.cat((x_ic, y_ic, z_ic, t_ic), dim=1)
    X_bc = torch.cat((x_bc, y_bc, z_bc, t_bc), dim=1)

    # Compute predictions
    u_pred, v_pred, w_pred = model(X_train).split(1, dim=1)

    u_ic_pred, v_ic_pred, w_ic_pred = model(X_ic).split(1, dim=1)

    loss_ic = torch.mean((u_ic_pred - u_ic) ** 2) + \
              torch.mean((v_ic_pred - v_ic) ** 2) + \
              torch.mean((w_ic_pred - w_ic) ** 2)

    u_bc_pred, v_bc_pred, w_bc_pred = model(X_bc).split(1, dim=1)

    loss_bc = torch.mean((u_bc_pred - u_bc_pred.detach()) ** 2) + \
              torch.mean((v_bc_pred - v_bc_pred.detach()) ** 2) + \
              torch.mean((w_bc_pred - w_bc_pred.detach()) ** 2)

    # PDE residual loss
    f_u, f_v, f_w = pde_residual(x_train, y_train, z_train, t_train, model)
    loss_pde = torch.mean(f_u**2 + f_v**2 + f_w**2)

    loss = loss_ic + loss_bc + loss_pde
    return loss

# Initialize model, optimizer
model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    loss = loss_function(model, x_train, y_train, z_train, t_train,
                         x_ic, y_ic, z_ic, t_ic, u_ic, v_ic, w_ic,
                         x_bc, y_bc, z_bc, t_bc)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
num_epochs = 12000
print("Hi")
for each in range(num_epochs):
    model.train()

    # Compute network output for interior points
    u_pred, v_pred, w_pred = model(x, y, z, t)

    # Compute network output for initial condition points
    u_ic_pred, v_ic_pred, w_ic_pred = model(x_ic, y_ic, z_ic, t_ic)

    # Compute loss for initial conditions
    loss_ic = torch.mean((u_ic_pred - u_ic) ** 2) + \
              torch.mean((v_ic_pred - v_ic) ** 2) + \
              torch.mean((w_ic_pred - w_ic) ** 2)

    # Compute network output for boundary condition points
    u_bc_pred, v_bc_pred, w_bc_pred = model(x_bc, y_bc, z_bc, t_bc)

    # Compute loss for boundary conditions (periodic BC)
    loss_bc = torch.mean((u_bc_pred - u_bc_pred.detach()) ** 2) + \
              torch.mean((v_bc_pred - v_bc_pred.detach()) ** 2) + \
              torch.mean((w_bc_pred - w_bc_pred.detach()) ** 2)


    # PDE residual loss
    f_u, f_v, f_w = pde_residual(x, y, t, model, nu)
    loss_pde = torch.mean(f_u**2 + f_v**2 + f_w**2)

    loss = loss_ic + loss_bc + loss_pde
    return loss


    # Compute initial condition loss
    #u_pred = model(x, y, torch.zeros_like(x))
    #u_true = initial_condition(x, y)
    #v_pred = model(x, y, torch.zeros_like(y))
    #v_true = initial_condition(x, y)
    #loss_ic = torch.mean((u_pred - u_true)**2 + (v_pred - v_true)**2)

    # Compute boundary condition loss
    #u_pred_left = model(torch.full_like(t, -1), t) # u(-1, t)
    #u_pred_right = model(torch.full_like(t, 1), t) # u(1, t)
    #v_pred_left = model(torch.full_like(t, -1), t) # u(-1, t)
    #v_pred_right = model(torch.full_like(t, 1), t) # u(1, t)
    #loss_bc = torch.mean((u_pred_left - boundary_condition(torch.full_like(t, -1), t))**2) + \
              #torch.mean((u_pred_right - boundary_condition(torch.full_like(t, 1), t))**2) + torch.mean((u_pred_left - boundary_condition(torch.full_like(t, -1), t))**2) + \
              #torch.mean((u_pred_right - boundary_condition(torch.full_like(t, 1), t))**2)

    # Compute PDE residual loss
    #residual = pde_residual(x_train, y_train, t_train, model)
    #loss_pde = torch.mean(residual**2)

    #loss = loss_ic + loss_bc + loss_pde

    # Initial condition loss
    u_pred, v_pred = model(x, y, torch.zeros_like(t))
    u0, v0 = initial_condition(x, y)
    loss_ic = torch.mean((u_pred - u0)**2 + (v_pred - v0)**2)

    # Boundary condition loss (assuming boundaries at x=±1, y=±1)
    u_pred_left, v_pred_left = model(torch.full_like(t, -1), y, t)
    u_pred_right, v_pred_right = model(torch.full_like(t, 1), y, t)
    u_pred_bottom, v_pred_bottom = model(x, torch.full_like(t, -1), t)
    u_pred_top, v_pred_top = model(x, torch.full_like(t, 1), t)
    u_bc, v_bc = boundary_condition(x, y, t)
    loss_bc = torch.mean((u_pred_left - u_bc)**2 + (u_pred_right - u_bc)**2 +
                         (u_pred_bottom - u_bc)**2 + (u_pred_top - u_bc)**2 +
                         (v_pred_left - v_bc)**2 + (v_pred_right - v_bc)**2 +
                         (v_pred_bottom - v_bc)**2 + (v_pred_top - v_bc)**2)

    # PDE residual loss
    f_u, f_v = pde_residual(x, y, t, model, nu)
    loss_pde = torch.mean(f_u**2 + f_v**2)

    loss = loss_ic + loss_bc + loss_pde

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     print("Hi")

    if each % 500 == 0:
        print(f'Epoch [{each + 1}/{num_epochs}], Loss: {loss.item():.4f}')




Hi


TypeError: PINN.forward() missing 1 required positional argument: 't'

In [ ]:
# evaluate the model on a grid of (x, t) values
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
x_test = torch.linspace(-1, 1, 100).view(-1, 1)
t_test = torch.linspace(0, 1, 100).view(-1, 1)
x_test, t_test  = torch.meshgrid(x_test.squeeze(), t_test.squeeze(), indexing = 'xy')
x_test = x_test.reshape(-1, 1)
t_test = t_test.reshape(-1, 1)

model.eval()
with torch.no_grad():
    u_pred = model(x_test, t_test).numpy()

# reshape the predicted u values for contour plotting
x_test = x_test.numpy().reshape(100, 100)
t_test = t_test.numpy().reshape(100, 100)
u_pred = u_pred.reshape(100, 100)

#plot the PINN solution as a contour plot
plt.figure(figsize = (16, 4))
plt.contourf(x_test, t_test, u_pred, levels = 250, cmap = 'jet')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('t')
plt.title('solution of burgers\' equation using PINNs')
plt.show()

#%%

#load the reference solution data
data= np.load("Burgers.npz")
t_ref, x_ref, exact = data["t"], data["x"], data["usol"].T
#Reshape
x_ref, t_ref = np.meshgrid(x_ref, t_ref)

# Define spatial and temporal grids
##x_ref = np.linspace(-1, 1, 100)  # 100 spatial points
#t_ref = np.linspace(0, 1, 100)  # 100 time points

# Example: Compute an analytical solution for visualization
#exact = np.sin(np.pi * x_ref[None, :]) * np.exp(-t_ref[:, None])

# Plot the reference solution as a contour plot
plt.figure(figsize=(16, 4))
plt.contourf(x_ref, t_ref, exact, levels=250, cmap='jet')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('t')
plt.title('Reference Solution of burgers\' equation')
plt.show()



In [ ]:

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (16, 6))

contour1 = ax1.contourf(x_test, t_test, u_pred, levels = 250, cmap = 'jet')
fig.colorbar(contour1, ax = ax1)
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_title('solution of burger\' equation uaing pinn (cpu)')


#plot reference
contour2 = ax2.contourf(x_ref, t_ref, exact, levels = 250, cmap = 'jet')
fig.colorbar(contour2, ax = ax2)
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_title('refernce solution of burger\' equation uaing pinn (cpu)')
#set same limits for both plots
ax1.set_xlim([x_ref.min(), x_ref.max()])
ax1.set_ylim([t_ref.min(), t_ref.max()])
ax2.set_xlim([x_ref.min(), x_ref.max()])
ax2.set_ylim([t_ref.min(), t_ref.max()])

plt.tight_layout()
plt.show()







In [ ]:

import matplotlib.pyplot as plt

time_slices = [ 0.25, 0.5, 0.75]
num_plots= len(time_slices)
cols = 2
rows = (num_plots // cols) +(num_plots % cols)
fig, axs = plt.subplots(rows, cols, figsize = (12, 5*rows))
axs = axs.flatten()
for i, t_val in enumerate(time_slices):
    # Find closest time point in t_ref
    idx_ref = np.argmin(np.abs(t_ref[:,0] - t_val))
    u_ref_slice = exact[idx_ref,:]

    t_slice = t_val * np.ones((100, 1))
    x_slice = np.linspace(-1, 1, 100).reshape(-1, 1)

    model.eval()
    with torch.no_grad():
        u_pinn_slice = model(torch.tensor(x_slice, dtype=torch.float32),
                             torch.tensor(t_slice, dtype=torch.float32)).numpy()

    axs[i].plot(x_ref[0, :], u_ref_slice, 'r-', label='Reference Solution')
    axs[i].plot(x_slice, u_pinn_slice, 'b--', label='PINN Solution')

    axs[i].set_xlabel('x')
    axs[i].set_ylabel('u')
    axs[i].set_title(f'solution at t = {t_val}')
    axs[i].legend()

for j in range(num_plots, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()